In [1]:
# Modules to import

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores.faiss import FAISS

import os
from dotenv import load_dotenv, find_dotenv

# Get OpenAI API KEY

In [2]:
load_dotenv(find_dotenv()) # read local .env file
api = os.getenv('OPENAI_API_KEY_H')

# Split Chunk

In [3]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

docs = []
for loader in os.listdir("docs"):
    if loader is not None :
        try :
            docs.extend(PyPDFLoader("docs/"+loader).load())
        except Exception as e1 :
            print(f" [ Documents ERROR ] : {e1}")
    else : break

In [4]:
docs[:2]

[Document(page_content='1-72 สารบัญ ภาพโดยรวม/คู่มืออย่างง่าย        1 ข้อมูลทั่วไป          2 การล็อกและการปลดล็อก        3 เบาะนั่งและเข็มขัดนิรภัย         4 แผงหน้าป้ดและอุปกรณ์ควบคุม        5 การสตาร์ทและการขับขี่         6 การใช้อุปกรณ์อํานวยความสะดวก       7 เมื่อเกิดเหตุฉุกเฉิน         8 การดูแลรักษาสภาพรถ         9 การบํารุงรักษา          10 ข้อมูลจําเพาะ          11 ', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 0}),
 Document(page_content='แผงหน้าป้ดและอุปกรณ์ควบคุม(ด้านคนขับ) ภาพโดยรวม/คู่มืออย่างง่าย1-11 - สวิตช์ที่ป้ดนํ้าฝนและฉีดนํ้าล้างกระจกหน้า5-45 - แผงหน้าป้ดหน้า5-2 - สวิตช์ควบคุมความสว่างของแผงหน้าป้ดหน้า5-2 - สวิตช์ระบบควบคุมเสถียรภาพการทรงตัว(ASC)OFFหน้า6-70 - สวิตช์เซ็นทรัลล็อกหน้า3-15 - สวิตช์ควบคุมหน้าต่างไฟฟ้าหน้า3-19 - สวิตช์ล็อกหน้า3-20 - สวิตช์ไฟหน้าและไฟสูง-ตํ่าหน้า5-40 - สวิตช์ระบบปรับไฟสูงอัตโนมัติ(AHB)หน้า5-43 - คันสวิตช์ไฟเลี้ยวหน้า5-44 - สวิตช์ไฟตัดหมอกด้านหน้าหน้า5-45 - สวิตช์หน้าจอแสดงข้อมูลรวมหน้า5-4 - สวิตช์ควบคุมเครื่องเสียงบนพวงมาลัยให้ดูคู่

# Vector Database

In [5]:
splits = r_splitter.split_documents(docs)

db = FAISS.from_documents( splits, OpenAIEmbeddings( api_key = api ) )

# CHAT

In [6]:
# Memory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
)

In [7]:
model_lists = [ "gpt-3.5-turbo", 'gpt-4-1106-preview', "gpt-4", ]

llm = ChatOpenAI( api_key = api, model_name = model_lists[0], temperature = 0 )

In [8]:
qa = ConversationalRetrievalChain.from_llm(
            llm = llm,
            retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2}),
            memory = memory,
            return_source_documents = True,
            return_generated_question = True,
        )

# Create Prompt

In [12]:
num_questions = 3
prompt = f"""
Create {num_questions} questions using only the context provided in the same language as documents.
End each question with a '?' character and then in a newline write the answer to that question using only the context provided.
Separate each question/answer pair by a newline.
"""

# Get answer

In [10]:
result = qa({"question": prompt})

In [25]:
print(f"Answer :\n{result['answer']}\n\nSource :\nPage : {(result['source_documents'])[0].metadata}\nContent : {(result['source_documents'])[0].page_content}")

Answer :
1. What does the display show when the "NORMAL" mode of the driving mode is selected? 
The display shows the cause and the solution to the problem. (Refer to page 6-47)

2. What does the display show when the "ROCK" mode of the driving mode is selected? 
The display shows the cause and the solution to the problem. (Refer to page 6-47)

3. What does the display show when the "MUD" mode of the driving mode is selected? 
The display shows the cause and the solution to the problem. (Refer to page 6-47)

Source :
Page : {'source': 'docs/Text_PDF_.pdf', 'page': 121}
Content : แผงหน้าปัดและอุปกรณ์ควบคุม 5  ไฟแสดงการทํางาน ไฟเตือน และรายการตัวแสดงบนหน้าจอแสดงข้อมูล (หน้าจอแสดงข้อมูลรวม - แบบที่ 1) 5-32  หน้าจอ: สาเหตุ: เมื่อเลือกโหมด “NORMAL” ของโหมดการขับเคลื่อน  วิธีแก้ป้ญหา (อ้างอิง): ให้ดูเรื่อง “โหมดการขับเคลื่อน” หน้า 6-47   หน้าจอ: สาเหตุ: เมื่อเลือกโหมด “ROCK” ของโหมดการขับเคลื่อน  วิธีแก้ป้ญหา (อ้างอิง): ให้ดูเรื่อง “โหมดการขับเคลื่อน” หน้า 6-47   หน้าจอ: สาเหตุ: เมื่อเลือกโห